# 3. Data ingestion

Using the python mysql connector, read and upload the previously generated CSV files into the MYSQL Database

Output/Deliverables:
- Table DDLs (Create table statements) in the folder sql/ddl/
- Python code that creates the tables and ingests the data into the tables

In [2]:
# INSTALL LIBRARIES
# ==============================================================================
# !pip install nbconvert
# !pip install python-dotenv
# !pip install mysql-connector
# !pip install mysql-connector-python
# pip install nbconvert

In [4]:
# IMPORT LIBRARIES

# Data processing
# ==============================================================================
import pandas as pd
import numpy as np

# mysql connector
# ==============================================================================
import mysql.connector 

# interface for SQLite databases
# ==============================================================================
import sqlite3

#Allows us to display more than one output per cell
# ==============================================================================
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# Display all columns
# ==============================================================================
pd.options.display.max_columns = None

# Hide passwords
# ==============================================================================
import os
from dotenv import load_dotenv
load_dotenv()

True

## File opening

In [5]:
# This is a function that reads the CSV file and returns the resulting DataFrame. 

def open_file (ubi,csv_file): #argument ubi is the folder where is the file, must be a string. `csv_file` which is expected to be a string representing the name of a CSV file.
    df = pd.read_csv(ubi+csv_file+'.csv', index_col=0) # The `index_col=0` argument specifies that the first column of the CSV file should be used as the index of the DataFrame.
    
    return df

``Teams``

In [6]:
df_teams = open_file("../sample/data/","teams")
df_teams.head()

,id,abbreviation,city,conference,division,full_name,name
0,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks
1,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
2,3,BKN,Brooklyn,East,Atlantic,Brooklyn Nets,Nets
3,4,CHA,Charlotte,East,Southeast,Charlotte Hornets,Hornets
4,5,CHI,Chicago,East,Central,Chicago Bulls,Bulls


``Games``

In [7]:
df_games = open_file("../sample/data/","games")
df_games.head()

,id,date,home_team_score,period,postseason,season,status,time,visitor_team_score,home_team.id,home_team.abbreviation,home_team.city,home_team.conference,home_team.division,home_team.full_name,home_team.name,visitor_team.id,visitor_team.abbreviation,visitor_team.city,visitor_team.conference,visitor_team.division,visitor_team.full_name,visitor_team.name
0,1164,1991-11-08T00:00:00.000Z,107,4,False,1991,Final,,101,27,SAS,San Antonio,West,Southwest,San Antonio Spurs,Spurs,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
1,1165,1991-11-09T00:00:00.000Z,118,4,False,1991,Final,,111,21,OKC,Oklahoma City,West,Northwest,Oklahoma City Thunder,Thunder,12,IND,Indiana,East,Central,Indiana Pacers,Pacers
2,1166,1992-04-15T00:00:00.000Z,95,4,False,1991,Final,,94,1,ATL,Atlanta,East,Southeast,Atlanta Hawks,Hawks,20,NYK,New York,East,Atlantic,New York Knicks,Knicks
3,1167,1992-04-16T00:00:00.000Z,96,4,False,1991,Final,,93,13,LAC,LA,West,Pacific,LA Clippers,Clippers,18,MIN,Minnesota,West,Northwest,Minnesota Timberwolves,Timberwolves
4,1168,1992-04-17T00:00:00.000Z,130,4,False,1991,Final,,106,21,OKC,Oklahoma City,West,Northwest,Oklahoma City Thunder,Thunder,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings


``Players``

In [8]:
df_players = open_file("../sample/data/","players")
df_players.head()

,id,first_name,height_feet,height_inches,last_name,position,weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
0,14,Ike,NaN,NaN,Anigbogu,C,NaN,12,IND,Indiana,East,Central,Indiana Pacers,Pacers
1,25,Ron,NaN,NaN,Baker,G,NaN,20,NYK,New York,East,Atlantic,New York Knicks,Knicks
2,47,Jabari,NaN,NaN,Bird,G,NaN,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
3,67,MarShon,NaN,NaN,Brooks,G,NaN,15,MEM,Memphis,West,Southwest,Memphis Grizzlies,Grizzlies
4,71,Lorenzo,NaN,NaN,Brown,G,NaN,28,TOR,Toronto,East,Atlantic,Toronto Raptors,Raptors


## Database creation

In [11]:
# os.getenv() is a method in the os module in Python that retrieves the value of an environment variable.
password_ada= os.getenv("password_ada")

In [12]:
# This function creates a new database in a MySQL server. 

def create_DB(nombre_bbdd): #`nombre_bbdd`, is the name of the database to be created, must be a string. 
    cnx = mysql.connector.connect(user='root', password=  password_ada,
                                host= '127.0.0.1') # A connection to the MySQL server is established using the `mysql.connector` module.

    mycursor = cnx.cursor() # A cursor object is created to execute SQL statements.

    sql = (f"CREATE DATABASE {nombre_bbdd}") # he SQL statement to create a new database is constructed using the `nombre_bbdd`argument.
    
    try: # Execute the SQL statement using the cursor's `execute()` method. If the execution is successful, a message is printed to confirm the creation of the database. 
        mycursor.execute(sql) 
        print(f"The database {nombre_bbdd} has been successfully created.")

    except mysql.connector.Error as err: # If an error occurs, the `except` block catches the error and prints the error message, error code, and SQL state. 
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    return mycursor.close() # Finally, the function closes the cursor using the `close()` method and returns `None`.
    

In [13]:
create_DB("db_ball_api")

1007 (HY000): Can't create database 'db_ball_api'; database exists
Error Code: 1007
SQLSTATE HY000
Message Can't create database 'db_ball_api'; database exists


True

## Tables creation

In [14]:
# This is a Python function that connects to a MySQL database using the `mysql.connector` module and executes a given SQL query. 

def execute_query(query): # The function takes a single argument `query`, which is the SQL query to be executed.
    
    cnx = mysql.connector.connect(user='root', password=password_ada,
                                host='127.0.0.1', database= 'db_ball_api')

    mycursor = cnx.cursor() # It creates a cursor object, executes the query using the cursor, and commits the changes to the database.


    try: # Execute the SQL statement using the cursor's `execute()` method.
        mycursor.execute(query)
        cnx.commit()

    except mysql.connector.Error as err: # If an error occurs, the `except` block catches the error and prints the error message, error code, and SQL state. 
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
    
    else:
        mycursor.close()
        cnx.close() # Finally, the function closes the cursor and the database connection.


`Teams`

In [62]:
with open('../sql/ddl/create_teams_table.sql', 'r') as sql_file:
    teams_sql_query = sql_file.read()
    execute_query (teams_sql_query)

In [63]:
teams_sql_query

'CREATE TABLE IF NOT EXISTS table_teams (\nid INT NOT NULL,\nabbreviation VARCHAR(255),\ncity VARCHAR(255),\nconference VARCHAR(255),\ndivision VARCHAR(255),\nfull_name VARCHAR(255),\n`name` VARCHAR(255),\nPRIMARY KEY (id));'

In [104]:
execute_query (teams_sql_query)

`Games`

In [18]:
with open('../sql/ddl/create_games_table.sql', 'r') as sql_file:
    games_sql_query = sql_file.read()

In [19]:
games_sql_query

'CREATE TABLE IF NOT EXISTS table_games (\nid INT NOT NULL ,\n`date` VARCHAR(255) ,\nhome_team_score INT ,\nperiod INT ,\npostseason VARCHAR(255) ,\nseason INT ,\nstatus VARCHAR(255) ,\ntime VARCHAR(255) ,\nvisitor_team_score INT ,\n`home_team.id` INT ,\n`home_team.abbreviation` VARCHAR(255) ,\n`home_team.city` VARCHAR(255) ,\n`home_team.conference` VARCHAR(255) ,\n`home_team.division` VARCHAR(255) ,\n`home_team.full_name` VARCHAR(255) ,\n`home_team.name` VARCHAR(255) ,\n`visitor_team.id` INT ,\n`visitor_team.abbreviation` VARCHAR(255) ,\n`visitor_team.city` VARCHAR(255) ,\n`visitor_team.conference` VARCHAR(255) ,\n`visitor_team.division` VARCHAR(255) ,\n`visitor_team.full_name` VARCHAR(255) ,\n`visitor_team.name` VARCHAR(255) ,\nPRIMARY KEY (id));'

In [108]:
execute_query (games_sql_query)

`Players`

In [21]:
with open('../sql/ddl/create_players_table.sql', 'r') as sql_file:
    players_sql_query = sql_file.read()

In [22]:
players_sql_query

'CREATE TABLE IF NOT EXISTS table_players (\nid INT NOT NULL ,\nfirst_name VARCHAR(255),\nheight_feet VARCHAR(255),\nheight_inches VARCHAR(255),\nlast_name VARCHAR(255),\nposition VARCHAR(255),\nweight_pounds VARCHAR(255),\n`team.id` INT,\n`team.abbreviation` VARCHAR(255),\n`team.city` VARCHAR(255),\n`team.conference` VARCHAR(255),\n`team.division` VARCHAR(255),\n`team.full_name` VARCHAR(255),\n`team.name` VARCHAR(255),\nPRIMARY KEY (id));'

In [23]:
execute_query (players_sql_query)

## Data ingestion

`Teams`

In [105]:
# This code is connecting to a SQLite database named 'DB_Ball_API', creating a cursor object, and then using pandas to write a dataframe named 'df_teams' to a table named 'table_teams' in the database. It then executes a SELECT statement to retrieve all rows from the 'table_teams' table, and for each row, it checks if any of the values are None. If any values are None, it replaces them with the string "None" and inserts the updated row into the 'table_teams' table using an INSERT statement. If none of the values are None, it inserts the original row into the 'table_teams' table using an INSERT statement. The function 'execute_query' is not shown in this code, so it is assumed to be defined elsewhere.

conn = sqlite3.connect('DB_Ball_API')
c = conn.cursor()

df_teams.to_sql('table_teams', conn, if_exists='replace', index = False)

c.execute('''  
SELECT * FROM table_teams
          ''')


for row in c.fetchall():

    #print (row)
    if None in list(row):
        list_none= list(row)
        for index, value in enumerate(list_none):
            if value == None:
                list_none[index] = "None"
            tupla_none=tuple(list_none)

        query_teams = f""" INSERT INTO table_teams (`id`, `abbreviation`, `city`, `conference`, `division`, `full_name`, `name`)
                            VALUES {tupla_none};
                                    """
        execute_query(query_teams)

    else: 
        query_teams = f""" INSERT INTO table_teams (`id`, `abbreviation`, `city`, `conference`, `division`, `full_name`, `name`)
                            VALUES {row};
                                    """
        execute_query(query_teams)


45

`Games`

In [109]:
# This code is connecting to a SQLite database named 'DB_Ball_API', creating a cursor object, and then using pandas to write a dataframe named 'df_games' to a table named 'table_games' in the database. It then executes a SELECT statement to retrieve all rows from the 'table_games' table, and for each row, it checks if any of the values are None. If any values are None, it replaces them with the string "None" and inserts the updated row into the 'table_games' table using an INSERT statement. If none of the values are None, it inserts the original row into the 'table_games' table using an INSERT statement. The function 'execute_query' is not shown in this code, so it is assumed to be defined elsewhere.

conn = sqlite3.connect('DB_Ball_API')
c = conn.cursor()

df_games.to_sql('table_games', conn, if_exists='replace', index = False)

c.execute('''  
SELECT * FROM table_games
          ''')


for row in c.fetchall():

    #print (row)
    if None in list(row):
        list_none= list(row)
        for index, value in enumerate(list_none):
            if value == None:
                list_none[index] = "None"
            tupla_none=tuple(list_none)

        query_games = f"""INSERT INTO table_games (`id`, `date`, `home_team_score`, `period`, `postseason`, `season`, `status`,
                                                    `time`, `visitor_team_score`,`home_team.id`, `home_team.abbreviation`, `home_team.city`,
                                                    `home_team.conference`, `home_team.division`, `home_team.full_name`, `home_team.name`, 
                                                    `visitor_team.id`, `visitor_team.abbreviation`, `visitor_team.city`, `visitor_team.conference`,
                                                    `visitor_team.division`, `visitor_team.full_name`, `visitor_team.name`)
                            
                            VALUES {tupla_none};
                                    """
        execute_query(query_games)

    else: 
        query_games = f"""INSERT INTO table_games (`id`, `date`, `home_team_score`, `period`, `postseason`, `season`, `status`,
                                                    `time`, `visitor_team_score`,`home_team.id`, `home_team.abbreviation`, `home_team.city`,
                                                    `home_team.conference`, `home_team.division`, `home_team.full_name`, `home_team.name`, 
                                                    `visitor_team.id`, `visitor_team.abbreviation`, `visitor_team.city`, `visitor_team.conference`,
                                                    `visitor_team.division`, `visitor_team.full_name`, `visitor_team.name`)
                            
                            VALUES {row};
                                    """
        execute_query(query_games)


1180

`Players`

In [110]:
# This code is connecting to a SQLite database named 'DB_Ball_API', creating a cursor object, and then using pandas to write a dataframe named 'df_games' to a table named 'table_games' in the database. It then executes a SELECT statement to retrieve all rows from the 'table_games' table, and for each row, it checks if any of the values are None. If any values are None, it replaces them with the string "None" and inserts the updated row into the 'table_games' table using an INSERT statement. If none of the values are None, it inserts the original row into the 'table_games' table using an INSERT statement. The function 'execute_query' is not shown in this code, so it is assumed to be defined elsewhere.

conn = sqlite3.connect('DB_Ball_API')
c = conn.cursor()

df_players.to_sql('table_players', conn, if_exists='replace', index = False)

c.execute('''  
SELECT * FROM table_players
          ''')


for row in c.fetchall():

    #print (row)
    if None in list(row):
        list_none= list(row)
        for index, value in enumerate(list_none):
            if value == None:
                list_none[index] = "None"
            tupla_none=tuple(list_none)

        query_players = f"""INSERT INTO table_players (`id`, `first_name`, `height_feet`, `height_inches`, `last_name`,
                                                    `position`, `weight_pounds`,`team.id`, `team.abbreviation`,
                                                    `team.city`, `team.conference`, `team.division`, `team.full_name`, 
                                                    `team.name`)
                            
                            VALUES {tupla_none};
                                    """
        execute_query(query_players)

    else: 
        query_players = f"""INSERT INTO table_players (`id`, `first_name`, `height_feet`, `height_inches`, `last_name`,
                                                    `position`, `weight_pounds`,`team.id`, `team.abbreviation`,
                                                    `team.city`, `team.conference`, `team.division`, `team.full_name`, 
                                                    `team.name`)
                            
                            VALUES {row};
                                    """
        execute_query(query_players)


5130

1062 (23000): Duplicate entry '46409591' for key 'table_players.PRIMARY'
Error Code: 1062
SQLSTATE 23000
Message Duplicate entry '46409591' for key 'table_players.PRIMARY'


## Clean null values

In [111]:
# This function creates a SQL query string that updates the specified `column` in the specified `table` to `NULL` where the value of the `column` is `'None'`. 

def update_to_null (table, column): # It takes two parameters: `table` and `column`.
    query = f"UPDATE {table} set {column} = NULL WHERE {column} = 'None'"
    execute_query(query) # The function then executes the query using a function called `execute_query`
    
    return query #returns the query as a string.

`Teams`

In [112]:
update_to_null("table_teams", "city")

"UPDATE table_teams set city = NULL WHERE city = 'None'"

In [113]:
update_to_null("table_teams", "division")

"UPDATE table_teams set division = NULL WHERE division = 'None'"

`Players`

In [114]:
update_to_null("table_players", "height_feet")

"UPDATE table_players set height_feet = NULL WHERE height_feet = 'None'"

In [115]:
update_to_null("table_players", "height_inches")

"UPDATE table_players set height_inches = NULL WHERE height_inches = 'None'"

In [116]:
update_to_null("table_players", "weight_pounds")

"UPDATE table_players set weight_pounds = NULL WHERE weight_pounds = 'None'"